This notebook takes an input sql file, parametrizes relevant queries and outputs an sql file containing a larger, parametrized set of queries

In [18]:
import re
import math
from datetime import datetime, date

Import sql queries

In [19]:
with open('tpch.txt') as file: #Todo- change this to a sql file..
    all_queries = file.read()

In [20]:
query_list = all_queries.split(';')
print(len(query_list))
print(query_list[5])

24


-- TPC-H 6
select
	sum(l_extendedprice * l_discount) as revenue
from
	snowflake_sample_data.tpch_sf1.lineitem
where
	l_shipdate >= date '1994-01-01'
	AND l_shipdate < DATEADD(year, 1, '1994-01-01')
	and l_discount between .06 - 0.01 and .06 + 0.01
	and l_quantity < 24


Parametrizing by dateadd functions

In [21]:
final_query_output = []
for query in query_list:
    #Split query by dateadd function into a list of query 'fragments'.
    arglist = re.split("dateadd", query, flags=re.IGNORECASE)
    
    #if there is at least one dateadd in the query...
    if len(arglist) > 1:
        for query_fragment in arglist[1:]:
            int_to_change = query_fragment.split(',')[1].strip() #first element will be the numeric argument of dateadd
            
            outer_index = query.index(query_fragment)
            inner_index = query_fragment.index(int_to_change)
            full_index = outer_index + inner_index
            
            #update the parameter and save off to a new query for each mod factor.
            for mod in [0.5, 1, 2]:
                modded_int = math.ceil(mod * int(int_to_change))
                modded_query = query[:full_index] + int_to_change + query[full_index+len(int_to_change):]
                final_query_output.append(modded_query)
                
    #no dateadd, so just add query
    else:
        final_query_output.append(query)
    
        
    arglist = re.split(" date ", query, flags=re.IGNORECASE)
    
    if len(arglist) > 1:
        for query_fragment in arglist[1:]:
            datetochange = query_fragment[1:11]
            initialDate = datetime.strptime(datetochange, '%Y-%m-%d')

            outer_index = query.index(query_fragment)
            inner_index = query_fragment.index(datetochange)
            full_index = outer_index + inner_index

            for mod in [0, 2, 4]:
                newDate = date(initialDate.year + (mod*2), initialDate.month, initialDate.day)
                newDate = str(newDate)
                modded_query = query[:full_index] + newDate + query[full_index+len(newDate):]
                final_query_output.append(modded_query)
    

In [22]:
print(len(final_query_output))

84


In [24]:
sql_string = ";\n".join([q for q in final_query_output])
#print(sql_string)

In [25]:
# Write to output SQL file... 
with open('output.sql', 'w') as outfile:
    outfile.write(sql_string)

Todo:
- Get rid of overlap in dateadds to prevent caching.
- Find other arguments to parametrize programatically